In [6]:
import hashlib as hasher
import datetime as date
import time
import random as rand
import ipyparallel as ipp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
#1 Blockchain Data Structure 
class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.nonce = nonce
        self.previous_hash = previous_hash
        self.hash = self.hash_block()


    def hash_block(self):

        sha = hasher.sha256()
        block_hash = (str(self.index) +
                      str(self.timestamp) +
                      str(self.data) +
                      str(self.previous_hash) +
                      str(self.nonce))
        block_hash = block_hash.encode('utf-8')
        sha.update(block_hash)
        return sha.hexdigest()



In [12]:
def block_validation(index, block_time, data, previous_hash):
    new_block = Block(index, block_time, data, previous_hash)
    check_string = '260b5039394689051b599484df495d79a6a33d22a3ca37af72656d8cdfc6fcd5'
    print_statement = "This is a valid Block" if str(new_block.hash) == check_string else "Please Check your work, this is incorrect."
    print(print_statement)
    
block_time = '2022-02-13 23:59:00'
data = 'Blockchain For Data Science'
previous_hash = '9136cfeb0c77b41e1e86cb9940ca9bb65f7aca4e8e366a8ecf9226b735e0c323'
index = 1
    
block_validation(index, block_time, data, previous_hash)

This is a valid Block


In [13]:
#2 Creating a chain out of single blocks
def create_genesis_block():
    return Block(0, date.datetime.now(), "Genesis Block", "0")

def next_block(last_block,nonce=0):
    this_index = last_block.index + 1
    this_timestamp = date.datetime.now()
    this_data = "Hey! I'm block " + str(this_index)
    this_prevhash = last_block.hash
    return Block(this_index, this_timestamp, this_data, this_prevhash)

In [17]:
def genesis_validation(genesis_block):
    block_1 = next_block(genesis_block)
    if block_1.index == 1 and block_1.data == "Hey! I'm block 1" and block_1.previous_hash == genesis_block.hash and str(type(block_1.timestamp)) == "<class 'datetime.datetime'>":
        print("Valid Genesis block" )
    else:
        print("Check the code, not a valid genesis block:(")

genesis_block = create_genesis_block()
genesis_validation(genesis_block)


Valid Genesis block


In [18]:
#3 Generating complete blockchain
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
num_blocks = 10

def complete_chain(num_blocks, blockchain, previous_block):
    for i in range(0, num_blocks):
        block_to_add = next_block(previous_block)
        blockchain.append(block_to_add)
        previous_block = block_to_add
        print("Block #{} has been added to the blockchain!".format(block_to_add.index))
        print("Hash: {}\n".format(block_to_add.hash))

complete_chain(num_blocks, blockchain, previous_block)

Block #1 has been added to the blockchain!
Hash: 593805e111af9c9450ce22dfe45ff0d62e45d2109873ef178c5b9ecc6c57c047

Block #2 has been added to the blockchain!
Hash: 3821e1881639e40e3e55a74fd1e2f634c6e65326a27a18537f1b6e38c5b141bd

Block #3 has been added to the blockchain!
Hash: 72d5f13bdaae1f67bed188f926aff9abf2d1a2b9fb0c239cd782b382207f511d

Block #4 has been added to the blockchain!
Hash: 5b5d6a9c6481a1dcd33c7be9e08772fcbdabd580b03249c7d084eef45ea406f0

Block #5 has been added to the blockchain!
Hash: cf18a1ccffe3a421b18ff0569c91e6f79b3e12db7ca0359dc8d41328eeccf81b

Block #6 has been added to the blockchain!
Hash: 90ed9d81eeb5d11a2b19f75410dec852ca0a1354dc5ab89b961e690c23784f94

Block #7 has been added to the blockchain!
Hash: 129614184d82932592ecf51a47fe5ccd7a970a02d88f8f0905ad62e361f80821

Block #8 has been added to the blockchain!
Hash: f09f5079d34a23dbaaf559af07a6e5293960510fc9d9219acb8278f04fe2ca7a

Block #9 has been added to the blockchain!
Hash: c2119ca55f0597aa9cf74c6bc0a3a0d

In [27]:
#4 Nonce and Difficulty

#4.1 Define function generate_nonce()
def generate_nonce(length=20):
    return ''.join([str(rand.randint(0, 9)) for i in range(length)])

#4.2 Define function generate_difficulty_bound()
def generate_difficulty_bound(difficulty=1):
    diff_str=""
    for i in range(0,difficulty):
        diff_str+="0"
    for i in range(0,64-difficulty):
        diff_str+="F"
    diff_str = "0x"+diff_str
    return(int(diff_str,16))

#4.3 Engineer a nonce given the previous block’s hash and difficulty
def find_next_block(last_block, difficulty, nonce_length):
    difficulty_bound = generate_difficulty_bound(difficulty)
    start = time.process_time()
    new_block = next_block(last_block)
    hashes_tried = 1
    while int(new_block.hash,16) > difficulty_bound:
        nonce = generate_nonce(nonce_length)
        new_block = Block(new_block.index, new_block.timestamp, new_block.data, new_block.previous_hash, nonce)
        hashes_tried += 1
    time_taken = time.process_time() - start   
    return time_taken, hashes_tried, new_block

In [28]:
#4.4 Blockchain with proof of work
blockchain = [create_genesis_block()]
previous_block = blockchain[0]
num_blocks = 20
difficulty = 3 
nonce_length = 10

def create_blockchain(num_blocks, difficulty, blockchain, previous_block, nonce_length, broadcast=1):
    hash_array = []
    time_array = []
    for i in range(0, num_blocks):
        time_taken, hashes_tried, block_to_add = find_next_block(previous_block, difficulty, nonce_length)
        blockchain.append(block_to_add)
        previous_block = block_to_add
        hash_array.append(hashes_tried)
        time_array.append(time_taken)
        if broadcast==1:
            print("Block #{} has been added to the blockchain!".format(block_to_add.index))
            print("{} Hashes Tried!".format(hashes_tried))
            print("Time taken to find block: {}".format(time_taken))
            print("Hash: {}\n".format(block_to_add.hash))     
    return(hash_array, time_array)

hash_array, time_array = create_blockchain(num_blocks, difficulty, blockchain, previous_block, nonce_length, broadcast=1)



Block #1 has been added to the blockchain!
535 Hashes Tried!
Time taken to find block: 0.0
Hash: 0004f759f848e79a1684048b36dfb88d08b808068498c07e5978a2ecd88df55f

Block #2 has been added to the blockchain!
1315 Hashes Tried!
Time taken to find block: 0.015625
Hash: 000bb8ae00f9d7faed9dc130aede822aa2d1af3adcd46def7c404031a781ccf9

Block #3 has been added to the blockchain!
1175 Hashes Tried!
Time taken to find block: 0.0
Hash: 0000f613fd235dd2095d07e0d2ce9c305700e396b1ec9b37c339112fee20c5e2

Block #4 has been added to the blockchain!
9142 Hashes Tried!
Time taken to find block: 0.046875
Hash: 000641f4ae77247201e6411a6ca3b484fa24070d6d0515ba97438080cdf7ee03

Block #5 has been added to the blockchain!
12190 Hashes Tried!
Time taken to find block: 0.125
Hash: 0004ade30294d6b3bb70e01eb1ad0e1e2f38a93e758be8e584910a57c711fb61

Block #6 has been added to the blockchain!
1213 Hashes Tried!
Time taken to find block: 0.015625
Hash: 000d8e8174085f72718b6086a085132111bb3a023e8db90107f6cf076c7f454c


In [30]:
def blockchain_proof(blockchain, num_blocks):
    correct = True
    bound = generate_difficulty_bound(difficulty)
    if len(blockchain) != num_blocks + 1:
        correct = False
    for i in range(len(blockchain) - 1):
        if blockchain[i + 1].previous_hash != blockchain[i].hash:
            correct = False
            break
        if int(blockchain[i + 1].hash, 16) > bound:
            correct = False
            break
    print_statement = "PASSED!!! Move on to the next Part" if correct else "FAILED!!! Try Again :("
    print(print_statement)
            
blockchain_proof(blockchain, num_blocks)

PASSED!!! Move on to the next Part


In [31]:
#5 Distributed Network

class MinerNodeNaive: 
    def __init__(self, name, compute):
        self.name = name 
        self.compute = compute
    
    def try_hash(self, diff_value, chain):
        last_block = chain[-1]
        difficulty = generate_difficulty_bound(diff_value)
        date_now = date.datetime.now()
        this_index = last_block.index + 1
        this_timestamp = date_now
        this_data = "Hey! I'm block " + str(this_index)
        this_hash = last_block.hash
        new_block = Block(this_index, this_timestamp, this_data, this_hash)
        if int(new_block.hash, 16) < difficulty:
            chain.append(new_block)
            # Tell everyone about it!
            print("Block #{} has been added to the blockchain!".format(new_block.index))
            print("Block found by: {}".format(self.name))
            print("Hash: {}\n".format(new_block.hash))


#This class MinerNodeNaive defines a simple, naive blockchain miner node that can attempt to mine new blocks and add them to a blockchain. 
#The class is initialized with a name attribute to identify the node and a compute attribute that doesn't seem to be utilized within the class.
#The try_hash method takes in a difficulty value and a blockchain as arguments, and attempts to mine a new block using various parameters including a new index (one greater than the index of the last block in the chain), a timestamp indicating the current time, some placeholder data, and the hash of the previous block in the chain.